In [1]:
import copy
import itertools as its
import math
import os
import pathlib
from typing import Dict, List, Optional
from collections import Counter, defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec11.txt'

In [17]:
class BoardInputWrapper:
    def __init__(self, robot):
        self.robot = robot
        
    def get(self):
        pos = (self.robot.pos_x, self.robot.pos_y)
        return self.robot.painted[pos]

    def __bool__(self):
        return True

class BoardOutputWrapper:
    def __init__(self, robot):
        self.robot = robot
        self.parity = 0
        self.has_been_painted = set()
    
    def __call__(self, val):
        if self.parity == 0:
          pos = (self.robot.pos_x, self.robot.pos_y)
          self.parity = 1
          self.robot.painted[pos] = val
          self.has_been_painted.add(pos)
        else:
          self.robot.turn(val)
          self.robot.step()
          self.parity = 0
        return False

class Robot:
    def __init__(self, ops):
        self.ops = ops
        self.direction = 0
        self.pos_x = 0
        self.pos_y = 0
        self.painted = defaultdict(int)
    
    def step(self):
        if self.direction == 0:
            self.pos_y -= 1
        elif self.direction == 2:
            self.pos_y += 1
        elif self.direction == 3:
            self.pos_x -= 1
        elif self.direction == 1:
            self.pos_x += 1
        else:
            raise ValueError(f'Invalid direction {self.direction}')
    
    def turn(self, val):
        if val == 0:
            # Turn left
            self.direction = (self.direction - 1) % 4
        elif val == 1:
            # Turn right
            self.direction = (self.direction + 1) % 4
        else:
            raise ValueError(f'Invalid turning direction {val}')
    
    def simulate(self, white_panels=None):
        input_wrapper = BoardInputWrapper(self)
        for pos in (white_panels or []):
            self.painted[pos] = 1
        output = BoardOutputWrapper(self)
        sim.simulate(self.ops, inputs=BoardInputWrapper(self), output_func=output)
        print(len(output.has_been_painted))
    
    def printme(self):
        min_x = min(key[0] for key in self.painted.keys())
        max_x = max(key[0] for key in self.painted.keys())
        min_y = min(key[1] for key in self.painted.keys())
        max_y = max(key[1] for key in self.painted.keys())
        for y in range(min_y, max_y + 1):
            for x in range(min_x + max_x + 1):
                print('.' if self.painted[(x, y)] == 0 else '#', end='')
            print()

In [18]:
robot = Robot(sim.read_ops(INPUT_PATH.read_text().strip()))
robot.simulate()

1909


In [19]:
robot.printme()

...........##............
...........#.###.........
..........#.##.#.........
........#..#.#.#.........
........#######.#........
....####.#.#...##........
....#..##.....#..........
...#.##.#..##.##.........
##.####..................
####..##....#..#.##......
.#....###.#.#######......
##.#...#...####..#.......
.###..####.#.##.####.....
.#.#......##...###.#.....
...##.#.#...###.#........
.#.##.####..#####.#......
#..#..#.#####....#.......
..#...#.##.#.#..#........
#..####.##.#..##.........
#..###..###..............
..##.###...##............
#..##......#.###.........
###..#.....#...#.........
#..#.#.##......#.........
.#.....##..#.##.#........
##.......##.##.##........
...#.##.###.#............
.#...#....####...........
###..#...#...............
.#...#..##.#.............
##.#......####.#.........
..##..#..##.#...##.......
#.#.#...#.####.#.#.......
.#.#.####....####........
#...##......#.#.#........
##.##.#.#...##.#.#.......
####.##.##.##..#.#.......
#####..##..##..###.......
....#..##..#

In [20]:
robot = Robot(sim.read_ops(INPUT_PATH.read_text().strip()))
robot.simulate(white_panels=[(0, 0)])

249


In [21]:
robot.printme()

...##.#..#.####.####.#..#.#..#.###..#..#...
....#.#..#.#....#....#.#..#..#.#..#.#..#...
....#.#..#.###..###..##...####.#..#.####...
....#.#..#.#....#....#.#..#..#.###..#..#...
.#..#.#..#.#....#....#.#..#..#.#....#..#...
..##...##..#....####.#..#.#..#.#....#..#...
